In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [3]:
L_set_base = general_cubic.get_L_set_in_plucker(['E1', 'G4', 'E2', 'G3', 'E3'])

In [4]:
possible_L_sets = [("E2", "G4", "E5", "G3", "E3"), ("E2", "G4", "E5", "G3", "F14"),\
           ("E2", "G4", "E5", "G3", "F46"),("E2", "G4", "E6", "G3", "E3"), \
	   ("E2", "G4", "E6", "G3", "F14"), ("E2", "G4", "E6", "G3", "F45"),\
           ("E2", "G4", "F34", "G3", "F14"), ("E2", "G4", "F34", "G3", "F45"),\
	   ("E2", "G4", "F34", "G3", "F46")]                 

In [16]:
%%time 
#projs = [find_projectivity(L_set_base, general_cubic.get_L_set_in_plucker(L_set_2)) for L_set_2 in possible_L_sets]
projs = general_cubic.find_all_proj_parallel(possible_L_sets)
projs_ideals = []
coeffs = []
sing_cubics_factored = sing_cubics.factor()
for proj in projs:
    print(projs.index(proj))
    #change coordinates, obtain new cubic and find conditions on the parameters
    sost = change_coord(proj)
    new_cubic = general_cubic.eqn.subs(sost)
    mon = (sum(P.gens()[0:4]) ^ 3).monomials()
    current_conds = list(set(matrix([[general_cubic.eqn.coefficient(mn) for mn in mon], [new_cubic.coefficient(mn) for mn in mon]]).minors(2)))
    no_sing_conds = [remove_sing_factors(el, sing_cubics_factored) for el in current_conds if el !=0]
    
    #two branches: assume e*f is zero and add to groebner basis, otherwise saturate with respect to e*f
    ideals = []
    gb = [remove_sing_factors(el, sing_cubics_factored) for el in P.ideal(no_sing_conds).groebner_basis()] 
    J = P.ideal(gb)
    ideals = J.saturation(ideal(c))[0].radical().primary_decomposition()
    
    #remove duplicates and singular ideals
    ideals = [ideal for ideal in ideals if sing_cubics not in ideal]
    
    # we obtain a list where, for each projectivity, there are the possible associated nonsingular ideals
    projs_ideals.append([proj, ideals])

0
1
2
3
4
5
6
7
8
CPU times: user 1min 3s, sys: 353 ms, total: 1min 3s
Wall time: 1min 3s


In [18]:
#number of ideals for each projectivity
[len(el[1]) for el in projs_ideals]

[1, 1, 1, 1, 1, 1, 1, 1, 1]

In [19]:
# here we obtain the matrices with the sostitutions obtained from the various ideals
matrices_from_ideals = []
all_matrices = []
for matrix_ideals in projs_ideals:
    proj = matrix_ideals[0]
    ideals = matrix_ideals[1]
    
    # matrices_from_ideals remembers from which ideals the matrices where generated
    matrices_from_ideals += [[matrix(4,4, [ideal.reduce(el) for el in proj.list()]), ideal] for ideal in ideals]

In [20]:
matrices_from_ideals_squared = [[el[0]^2, el[1]] for el in matrices_from_ideals]
identity_list = [0 for _ in range(16)]
for i in {0,5,10,15}:
    identity_list[i]=1
id_mat = matrix(4,4,identity_list)
len(matrices_from_ideals_squared)

9

In [21]:
for mat_id in matrices_from_ideals_squared:
    print(are_matrices_equal(matrix(4,4,[mat_id[1].reduce(el) for el in mat_id[0].list()]), id_mat))

False
False
False
False
False
False
False
False
False


In [22]:
final_matrices = []
final_ideals = []
for mat_id in matrices_from_ideals_squared:
    final_matrices.append(matrix(4,4, [mat_id[1].reduce(el) for el in mat_id[0].list()]))
    final_ideals.append(mat_id[1])

In [23]:
for mat_id in matrices_from_ideals_squared:
    print(are_matrices_equal(matrix(4,4,[mat_id[1].reduce(el) for el in (mat_id[0]^2).list()]), id_mat))

False
False
False
False
False
False
False
False
False


In [24]:
cubics = [general_cubic.reduce(ideal) for ideal in final_ideals]

In [25]:
[len(cubic.eckardt_points_labels) for cubic in cubics]

[18, 10, 10, 18, 10, 10, 18, 10, 10]